In [0]:
%run "../common functions"

In [0]:
spark.sql('truncate table staging_catalogue.jobs.linkedjobs')
all_jobs = spark.table("raw_catalogue.jobs.linkedjobs")
jobs = all_jobs.collect()

In [0]:
scored_jobs = score_job(jobs, "url", "description", "skills")

In [0]:
scored_df = spark.createDataFrame(scored_jobs)
df = scored_df.join(all_jobs, on=["url"])
filtered_df = df.filter(df.score > 60)
filtered_df.write.mode('overwrite').option("mergeSchema", "true").saveAsTable("staging_catalogue.jobs.linkedJobs")

In [0]:
display(filtered_df)